# Food Dude
### Jaiden Gerig, Oron Hazi, Justin Katz, Kyle Wilson

## Importing Our Data
Luckily this is all in a CSV file so we can grab it easilty with Pandas

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

foods = pd.read_csv('ABBREV.csv', index_col=0)
foods.head()

,Shrt_Desc,Water_(g),Energ_Kcal,Protein_(g),Lipid_Tot_(g),Ash_(g),Carbohydrt_(g),Fiber_TD_(g),Sugar_Tot_(g),Calcium_(mg),...,Vit_K_(�g),FA_Sat_(g),FA_Mono_(g),FA_Poly_(g),Cholestrl_(mg),GmWt_1,GmWt_Desc1,GmWt_2,GmWt_Desc2,Refuse_Pct
NDB_No,,,,,,,,,,,,,,,,,,,,,
1001,"BUTTER,WITH SALT",15.87,717,0.85,81.11,2.11,0.06,0.0,0.06,24.0,...,7.0,51.368,21.021,3.043,215.0,5.00,"1 pat, (1"" sq, 1/3"" high)",14.2,1 tbsp,0.0
1002,"BUTTER,WHIPPED,W/ SALT",16.72,718,0.49,78.30,1.62,2.87,0.0,0.06,23.0,...,4.6,45.390,19.874,3.331,225.0,3.80,"1 pat, (1"" sq, 1/3"" high)",9.4,1 tbsp,0.0
1003,"BUTTER OIL,ANHYDROUS",0.24,876,0.28,99.48,0.00,0.00,0.0,0.00,4.0,...,8.6,61.924,28.732,3.694,256.0,12.80,1 tbsp,205.0,1 cup,0.0
1004,"CHEESE,BLUE",42.41,353,21.40,28.74,5.11,2.34,0.0,0.50,528.0,...,2.4,18.669,7.778,0.800,75.0,28.35,1 oz,17.0,1 cubic inch,0.0
1005,"CHEESE,BRICK",41.11,371,23.24,29.68,3.18,2.79,0.0,0.51,674.0,...,2.5,18.764,8.598,0.784,94.0,132.00,"1 cup, diced",113.0,"1 cup, shredded",0.0


## Washing Our Food
to start off we only wanted to look at the main nutritional values (Protein,Fat,Sodium,Etc.) of each food

In [2]:
#Rename some of our columns to something a bit easier on the eyes
foods = foods.rename(index=str,columns={'Shrt_Desc':'Name','Protein_(g)':'Protein (g)','Lipid_Tot_(g)':'Total Fat(g)','Cholestrl_(mg)':'Cholesterol (mg)',
               'FA_Sat_(g)':'Saturated Fat (g)','Sodium_(mg)':'Sodium (mg)','Potassium_(mg)':'Potassium (mg)',
               'Carbohydrt_(g)':'Carbohydrates (g)','Fiber_TD_(g)':'Fiber (g)'})
# Look at a specific subset of nutrients
foods['Weight (g)'] = 100
nutrients = ['Name','Protein (g)','Total Fat(g)','Cholesterol (mg)',
               'Saturated Fat (g)','Sodium (mg)','Potassium (mg)',
               'Carbohydrates (g)','Fiber (g)','Weight (g)']
foods = foods[nutrients]
foods = foods.fillna(0)
# Get rid of foods we dont have serving sizes for
foods = foods[foods.apply(lambda x:x['Weight (g)'] > 0, axis=1)]
foods.head()

,Name,Protein (g),Total Fat(g),Cholesterol (mg),Saturated Fat (g),Sodium (mg),Potassium (mg),Carbohydrates (g),Fiber (g),Weight (g)
NDB_No,,,,,,,,,,
1001,"BUTTER,WITH SALT",0.85,81.11,215.0,51.368,643.0,24.0,0.06,0.0,100
1002,"BUTTER,WHIPPED,W/ SALT",0.49,78.30,225.0,45.390,583.0,41.0,2.87,0.0,100
1003,"BUTTER OIL,ANHYDROUS",0.28,99.48,256.0,61.924,2.0,5.0,0.00,0.0,100
1004,"CHEESE,BLUE",21.40,28.74,75.0,18.669,1146.0,256.0,2.34,0.0,100
1005,"CHEESE,BRICK",23.24,29.68,94.0,18.764,560.0,136.0,2.79,0.0,100


We wanted to look the nutritional value of each food regardless of serving size, so we normalized them to 1 gram

In [3]:
# Normalizing all our foods to 1 g
def normalizeNutrients(x):
    ratio = x['Weight (g)']
    for nutrient in nutrients:
        if(type(x[nutrient]) is str):
            continue
        x[nutrient] = x[nutrient]/ratio
    return x
foods = foods.apply(normalizeNutrients,axis=1)
avg_nutrients = foods.mean()
avg_nutrients = avg_nutrients.drop("Weight (g)")
foods.head()

,Name,Protein (g),Total Fat(g),Cholesterol (mg),Saturated Fat (g),Sodium (mg),Potassium (mg),Carbohydrates (g),Fiber (g),Weight (g)
NDB_No,,,,,,,,,,
1001,"BUTTER,WITH SALT",0.0085,0.8111,2.15,0.51368,6.43,0.24,0.0006,0.0,1
1002,"BUTTER,WHIPPED,W/ SALT",0.0049,0.7830,2.25,0.45390,5.83,0.41,0.0287,0.0,1
1003,"BUTTER OIL,ANHYDROUS",0.0028,0.9948,2.56,0.61924,0.02,0.05,0.0000,0.0,1
1004,"CHEESE,BLUE",0.2140,0.2874,0.75,0.18669,11.46,2.56,0.0234,0.0,1
1005,"CHEESE,BRICK",0.2324,0.2968,0.94,0.18764,5.60,1.36,0.0279,0.0,1


## Making a Meal
We started our food analysis by taking all the foods and Scaling them up to meet the demands of an average a 2000 calorie diet according to [Netrition](http://www.netrition.com/rdi_page.html).

The first step we took was to exclude foods that didnt contain all the nutrients we were looking at

In [4]:
# We only want foods that have a chance to sustain our needs
def filterNutrients(x):
    for nutrient in nutrients:
        if(x[nutrient] <= 0):
            return False
    return True
foods = foods[foods.apply(filterNutrients, axis=1)]
print 'Matching Foods:',len(foods)
foods.head()

Matching Foods: 1425


,Name,Protein (g),Total Fat(g),Cholesterol (mg),Saturated Fat (g),Sodium (mg),Potassium (mg),Carbohydrates (g),Fiber (g),Weight (g)
NDB_No,,,,,,,,,,
1013,"CHEESE,COTTAGE,CRMD,W/FRUIT",0.1069,0.0385,0.13,0.02311,3.44,0.90,0.0461,0.002,1
1043,"CHEESE,PAST PROCESS,PIMENTO",0.2213,0.3120,0.94,0.19663,9.15,1.62,0.0173,0.001,1
1102,"MILK,CHOC,FLUID,COMM,WHL,W/ ADDED VIT A & VITA...",0.0317,0.0339,0.12,0.02104,0.60,1.67,0.1034,0.008,1
1103,"MILK,CHOC,FLUID,COMM,RED FAT",0.0299,0.0190,0.08,0.01177,0.66,1.69,0.1213,0.007,1
1104,"MILK,CHOC,LOWFAT,W/ ADDED VIT A & VITAMIN D",0.0346,0.0100,0.05,0.00584,0.65,1.72,0.0986,0.001,1


Nice, That leaves us with almost 1400 foods to look at, not too shabby
So let's take these foods and scale them up to see how many grams of each we would have to consume to fulfill our daily nutrition requirements

In [5]:
# http://www.netrition.com/rdi_page.html
recommended = [-1,50,65,300,20,2400,3500,300,25,-1]
def findSatisfyingWeight(food):
    for x in range(0,len(nutrients)):
        nutrient = nutrients[x]
        rec = recommended[x]
        if(rec == -1 or food[nutrient] >= rec):
            continue
        ratio = rec/food[nutrient]
        for y in nutrients:
            if(type(food[y]) is str):
                continue
            food[y] = food[y]*ratio
    return food   
weighted_foods = foods.apply(findSatisfyingWeight,axis=1)
display = ['Name','Weight (g)','Protein (g)','Total Fat(g)','Cholesterol (mg)',
               'Saturated Fat (g)','Sodium (mg)','Potassium (mg)',
               'Carbohydrates (g)','Fiber (g)']
weighted_foods[display].sort_values(by='Weight (g)').head(10)

,Name,Weight (g),Protein (g),Total Fat(g),Cholesterol (mg),Saturated Fat (g),Sodium (mg),Potassium (mg),Carbohydrates (g),Fiber (g)
NDB_No,,,,,,,,,,
11672,POTATO PANCAKES,1078.748652,65.587918,159.223301,1024.811219,26.925566,8241.639698,6709.816613,300.000000,35.598706
6128,"SOUP,CHICK NOODLE,DRY,MIX",1143.790850,176.372549,74.460784,846.405229,22.521242,41668.300654,3500.000000,712.810458,36.601307
21604,"SCHOOL LUNCH,PIZZA,SSAGE TOPPING,THIN CRST,WHL...",1200.000000,151.560000,93.960000,300.000000,41.856000,5484.000000,4620.000000,386.880000,46.800000
21605,"SCHOOL LUNCH,PIZZA,SAUSAGE TOP,THICK CRUST,WHL...",1241.134752,164.574468,112.570922,322.695035,52.363475,5374.113475,3500.000000,379.539007,49.645390
6959,"GRAVY,INST TURKEY,DRY",1250.000000,146.500000,183.250000,300.000000,61.637500,51125.000000,3825.000000,719.500000,47.500000
21602,"SCHOOL LUNCH,PIZZA,PEPPI TOPPING,THIN CRUST,WH...",1250.000000,159.750000,107.750000,300.000000,35.125000,6187.500000,4775.000000,390.500000,53.750000
21603,"SCHOOL LUNCH,PIZZA,PEPP TOPPING,THICK CRUST,WH...",1250.000000,179.375000,122.750000,300.000000,52.737500,5925.000000,3850.000000,353.750000,53.750000
18300,"PANCAKES,WHOLE-WHEAT,DRY MIX,INCOMPLETE,PREP",1254.480287,106.630824,81.541219,765.232975,21.940860,7175.627240,3500.000000,368.817204,35.125448
43260,"BEVERAGE,INST BRKFST PDR,CHOC,SUGAR-FREE,NOT R...",1274.509804,456.274510,65.000000,560.784314,27.554902,9138.235294,21730.392157,522.549020,25.490196


Yikes, none of that seems very healthy at all

To some of these foods into perspective:

* Potato Pancakes: 1078 grams = 49 pancakes
* Chicken Noodle Soup: 1143 grams = 15 packets
* Digiorno Thin Crust Pizza: 1305 grams = 2.4 Pizza

Let's see what happens when we look at how far over the reccomended nutritional values these foods go

In [6]:
from bokeh.io import push_notebook,show,output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.charts import Bar, output_file, show
from bokeh.models import Range1d
from bokeh.charts.operations import blend
from bokeh import palettes
output_notebook()
def findOverages(food):
    for x in range(0,len(nutrients)):
        nutrient = nutrients[x]
        rec = recommended[x]
        if(rec == -1):
            continue
        food[nutrient] -= rec
    return food   
overage_foods = weighted_foods.apply(findOverages,axis=1)
df = overage_foods.sort_values(by='Weight (g)').head(10)
a = Bar(df, 'Name', values='Protein (g)', title="Excess Protein",legend=False,width=450,continuous_range=Range1d(0,100))
b = Bar(df, 'Name', values='Total Fat(g)', title="Excess Total Fat",legend=False,width=450)
a.xaxis.axis_label = ""
b.xaxis.axis_label = ""
show(row(a,b))
a = Bar(df, 'Name', values='Cholesterol (mg)', title="Excess Cholesterol",legend=False,width=450)
b = Bar(df, 'Name', values='Saturated Fat (g)', title="Excess Saturated Fats",legend=False,width=450)
a.xaxis.axis_label = ""
b.xaxis.axis_label = ""
show(row(a,b))
a = Bar(df, 'Name', values='Sodium (mg)', title="Excess Sodium",legend=False,width=450)
b = Bar(df, 'Name', values='Potassium (mg)', title="Excess Potassium",legend=False,width=450)
a.xaxis.axis_label = ""
b.xaxis.axis_label = ""
show(row(a,b))
a = Bar(df, 'Name', values='Carbohydrates (g)', title="Excess Carbohydrates",legend=False,width=450)
b = Bar(df, 'Name', values='Fiber (g)', title="Excess Fiber",legend=False,width=450)
a.xaxis.axis_label = ""
b.xaxis.axis_label = ""
show(row(a,b))

Loading BokehJS ...

So that gives us a general idea of what kind of foods have common excesses but it's hard to compare them to each-other so let's convert them to percentages of the 2000 calorie diet over 100%

In [7]:
def findPercentOverages(food):
    for x in range(0,len(nutrients)):
        nutrient = nutrients[x]
        rec = recommended[x]
        if(rec == -1):
            continue
        food[nutrient] = ((food[nutrient]/rec)*100)-100
    return food   
overage_foods = weighted_foods.apply(findPercentOverages,axis=1)
overage_foods[display].sort_values(by='Weight (g)').head(10).rename(index=str,columns={'Protein (g)':'Protein (%)','Total Fat(g)':'Total Fat(%)','Cholesterol (mg)':'Cholesterol (%)',
               'Saturated Fat (g)':'Saturated Fat (%)','Sodium (mg)':'Sodium (%)','Potassium (mg)':'Potassium (%)',
               'Carbohydrates (g)':'Carbohydrates (%)','Fiber (g)':'Fiber (g)'})

,Name,Weight (g),Protein (%),Total Fat(%),Cholesterol (%),Saturated Fat (%),Sodium (%),Potassium (%),Carbohydrates (%),Fiber (g)
NDB_No,,,,,,,,,,
11672,POTATO PANCAKES,1078.748652,31.175836,144.958925,241.603740,34.627832,243.401654,9.170905e+01,0.000000,42.394822
6128,"SOUP,CHICK NOODLE,DRY,MIX",1143.790850,252.745098,14.555053,182.135076,12.606209,1636.179194,0.000000e+00,137.603486,46.405229
21604,"SCHOOL LUNCH,PIZZA,SSAGE TOPPING,THIN CRST,WHL...",1200.000000,203.120000,44.553846,0.000000,109.280000,128.500000,3.200000e+01,28.960000,87.200000
21605,"SCHOOL LUNCH,PIZZA,SAUSAGE TOP,THICK CRUST,WHL...",1241.134752,229.148936,73.186034,7.565012,161.817376,123.921395,0.000000e+00,26.513002,98.581560
6959,"GRAVY,INST TURKEY,DRY",1250.000000,193.000000,181.923077,0.000000,208.187500,2030.208333,9.285714e+00,139.833333,90.000000
21602,"SCHOOL LUNCH,PIZZA,PEPPI TOPPING,THIN CRUST,WH...",1250.000000,219.500000,65.769231,0.000000,75.625000,157.812500,3.642857e+01,30.166667,115.000000
21603,"SCHOOL LUNCH,PIZZA,PEPP TOPPING,THICK CRUST,WH...",1250.000000,258.750000,88.846154,0.000000,163.687500,146.875000,1.000000e+01,17.916667,115.000000
18300,"PANCAKES,WHOLE-WHEAT,DRY MIX,INCOMPLETE,PREP",1254.480287,113.261649,25.448029,155.077658,9.704301,198.984468,2.842171e-14,22.939068,40.501792
43260,"BEVERAGE,INST BRKFST PDR,CHOC,SUGAR-FREE,NOT R...",1274.509804,812.549020,0.000000,86.928105,37.774510,280.759804,5.208683e+02,74.183007,1.960784


And now let's chart them again with a uniform scale


In [8]:
df = overage_foods.sort_values(by='Weight (g)').head(10).rename(index=str,columns={'Protein (g)':'Protein (%)','Total Fat(g)':'Total Fat(%)','Cholesterol (mg)':'Cholesterol (%)',
               'Saturated Fat (g)':'Saturated Fat (%)','Sodium (mg)':'Sodium (%)','Potassium (mg)':'Potassium (%)',
               'Carbohydrates (g)':'Carbohydrates (%)','Fiber (g)':'Fiber (g)'})
a = Bar(df, label='vars',group='Name', 
        values=blend('Protein (%)', 'Total Fat(%)','Cholesterol (%)',
                     'Saturated Fat (%)','Sodium (%)','Potassium (%)',
                     'Carbohydrates (%)','Fiber (%)',name='values', labels_name='vars'),
        title="Excess Nutrients (% above recommended daily intake)",width=900)
a.xaxis.axis_label = ""
a.yaxis.axis_label = "% above reccomended daily intake"
show(a)
output_notebook()

Loading BokehJS ...

Holy Guacamole! Look at that sodium!

It looks like potassium and sodium are crazy high compared to the other nutrients, which is most likely because they're the only nutrients measured in milligrams instead of grams, so let's see what the graph looks like without them so we can get a better understanding of the other nutrients

In [9]:
df = overage_foods.sort_values(by='Weight (g)').head(10).rename(index=str,columns={'Protein (g)':'Protein (%)','Total Fat(g)':'Total Fat(%)','Cholesterol (mg)':'Cholesterol (%)',
               'Saturated Fat (g)':'Saturated Fat (%)','Sodium (mg)':'Sodium (%)','Potassium (mg)':'Potassium (%)',
               'Carbohydrates (g)':'Carbohydrates (%)','Fiber (g)':'Fiber (g)'})
a = Bar(df, label='vars',group='Name', 
        values=blend('Protein (%)', 'Total Fat(%)','Cholesterol (%)',
                     'Saturated Fat (%)',
                     'Carbohydrates (%)','Fiber (%)',name='values', labels_name='vars'),
        title="Excess Nutrients (% above recommended daily intake) (Excluding Sodium & Potassium)",width=900,height=1000,palette=palettes.BrBG11)
a.xaxis.axis_label = ""
a.yaxis.axis_label = "% above reccomended daily intake"
show(a)
output_notebook()

Loading BokehJS ...

So overall, it appears that these foods are providing mainly carbs and sodium at the expense of other nutrients 

## What about nutrients per calorie?

As most people know, the average optimal number of calories that someone should eat in one day is 2000. The work we have done thus far is concerned with the amount of nutrients per gram, but grams are not necessarily the best indication of the nutritional value of your food. Because of this, we want to do similar analysis, but optimizing the nutrients per calorie, rather than nutrients per gram.

In [10]:
# Here we do some preliminary data preparation, mostly copying what was done before.

calfoods = pd.read_csv('ABBREV.csv', index_col=0)

#Rename some of our columns to something a bit easier on the eyes
calfoods = calfoods.rename(index=str,columns={'Shrt_Desc':'Name','Protein_(g)':'Protein (g)','Lipid_Tot_(g)':'Total Fat(g)','Cholestrl_(mg)':'Cholesterol (mg)',
               'FA_Sat_(g)':'Saturated Fat (g)','Sodium_(mg)':'Sodium (mg)','Potassium_(mg)':'Potassium (mg)',
               'Carbohydrt_(g)':'Carbohydrates (g)','Fiber_TD_(g)':'Fiber (g)','Energ_Kcal':'Calories'})
# Look at a specific subset of nutrients
calnutrients = ['Name','Protein (g)','Total Fat(g)','Cholesterol (mg)',
               'Saturated Fat (g)','Sodium (mg)','Potassium (mg)',
               'Carbohydrates (g)','Fiber (g)','Calories']
calfoods = calfoods[calnutrients]
calfoods = calfoods.fillna(0)
# Get rid of foods we dont have serving sizes for
calfoods = calfoods[calfoods.apply(lambda x:x['Calories'] > 0, axis=1)]

# Convert sodium and potassium to grams


# Normalizing all our foods to 1 calorie
def normalizeNutrientsCal(x):
    ratio = x['Calories']
    for nutrient in calnutrients:
        if(type(x[nutrient]) is str):
            continue
        x[nutrient] = x[nutrient]/ratio
    return x
calfoods = calfoods.apply(normalizeNutrientsCal,axis=1)
calfoods.head()

# We only want foods that have a chance to sustain our needs
def filterNutrientsCal(x):
    for nutrient in calnutrients:
        if(x[nutrient] <= 0):
            return False
    return True
calfoods = calfoods[calfoods.apply(filterNutrientsCal, axis=1)]
calfoods.head(5)

,Name,Protein (g),Total Fat(g),Cholesterol (mg),Saturated Fat (g),Sodium (mg),Potassium (mg),Carbohydrates (g),Fiber (g),Calories
NDB_No,,,,,,,,,,
1013,"CHEESE,COTTAGE,CRMD,W/FRUIT",0.110206,0.039691,0.134021,0.023825,3.546392,0.927835,0.047526,0.002062,1
1043,"CHEESE,PAST PROCESS,PIMENTO",0.059013,0.083200,0.250667,0.052435,2.440000,0.432000,0.004613,0.000267,1
1102,"MILK,CHOC,FLUID,COMM,WHL,W/ ADDED VIT A & VITA...",0.038193,0.040843,0.144578,0.025349,0.722892,2.012048,0.124578,0.009639,1
1103,"MILK,CHOC,FLUID,COMM,RED FAT",0.039342,0.025000,0.105263,0.015487,0.868421,2.223684,0.159605,0.009211,1
1104,"MILK,CHOC,LOWFAT,W/ ADDED VIT A & VITAMIN D",0.055806,0.016129,0.080645,0.009419,1.048387,2.774194,0.159032,0.001613,1


As we can see, we now have all of the foods with their nutrients weighted per calorie. However, this does not tell us a whole lot. Lets find out how many calories we would need to eat for each food to get our nutritional goal.

In [11]:
# http://www.netrition.com/rdi_page.html
recommended = [-1,50,65,300,20,2400,3500,300,25,-1]
def findSatisfyingWeightCal(food):
    for x in range(0,len(calnutrients)):
        nutrient = calnutrients[x]
        rec = recommended[x]
        if(rec == -1 or food[nutrient] >= rec):
            continue
        ratio = rec/food[nutrient]
        for y in calnutrients:
            if(type(food[y]) is str):
                continue
            food[y] = food[y]*ratio
    return food   
calweighted_foods = calfoods.apply(findSatisfyingWeightCal,axis=1)
display = ['Name','Calories','Protein (g)','Total Fat(g)','Cholesterol (mg)',
               'Saturated Fat (g)','Sodium (mg)','Potassium (mg)',
               'Carbohydrates (g)','Fiber (g)']
calweighted_foods[display].sort_values(by='Calories').head(10)

,Name,Calories,Protein (g),Total Fat(g),Cholesterol (mg),Saturated Fat (g),Sodium (mg),Potassium (mg),Carbohydrates (g),Fiber (g)
NDB_No,,,,,,,,,,
3127,"BABYFOOD,VEG,SPINACH,CRMD,STR",2220.000000,150.000000,78.000000,300.000000,42.120000,2940.000000,11460.000000,342.000000,108.000000
6082,"CAMPBELL'S RED & WHITE,CHICK NOODLEO'S SOUP,COND",2330.808081,78.131313,65.000000,525.252525,26.065657,12507.575758,14083.333333,390.656566,26.262626
16059,"CHILI WITH BEANS,CANNED",2333.836858,138.670695,85.196375,385.196375,25.672205,9584.592145,8270.392749,300.000000,74.773414
6986,"CAMPBELL'S HEALTHY REQUEST,CHICK NOODLE SOUP,COND",2350.000000,120.000000,65.000000,500.000000,20.000000,16700.000000,17500.000000,325.500000,50.000000
22982,"KASHI,STEAM MEAL,CHICK FETTUCCINE,FRZ ENTREE",2383.333333,149.259259,65.000000,337.037037,21.666667,3996.296296,4044.444444,337.037037,48.148148
11372,"POTATOES,SCALLPD,HOME-PREPARED W/BUTTER",2448.979592,79.870130,102.411874,333.951763,62.755102,9322.820037,10519.480519,300.000000,52.875696
11387,"POTATOES,SCALLPD,DRY MIX,PREP W/H2O,WHL MILK&B...",2536.363636,57.818182,117.272727,300.000000,71.809091,9300.000000,5536.363636,348.272727,30.000000
3296,"BABYFOOD,TURKEY,RICE&VEG,TODD",2571.428571,162.857143,68.571429,300.000000,21.428571,7842.857143,4585.714286,321.428571,34.285714
11385,"POTATOES,AU GRATIN,DRY MIX,PREP W/H2O,WHL MILK...",2583.333333,63.888889,114.444444,416.666667,71.833333,12194.444444,6083.333333,356.666667,25.000000


Like before, we go far over our goals for each nutrient. We also go over our preferred caloric intake. Additonally, we get a very different set of foods than when we looked at nutrients per gram. Let's now look and see how far over each of these we are going.

In [12]:
from bokeh.io import push_notebook,show,output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.charts import Bar, output_file, show
from bokeh.models import Range1d
from bokeh.charts.operations import blend
from bokeh import palettes
def findOveragesCal(food):
    for x in range(0,len(calnutrients)):
        nutrient = calnutrients[x]
        rec = recommended[x]
        if(rec == -1):
            continue
        food[nutrient] -= rec
    return food   
caloverage_foods = calweighted_foods.apply(findOveragesCal,axis=1)
df = caloverage_foods.sort_values(by='Calories').head(10)
a = Bar(df, 'Name', values='Protein (g)', title="Excess Protein",legend=False,width=450,continuous_range=Range1d(0,100))
b = Bar(df, 'Name', values='Total Fat(g)', title="Excess Total Fat",legend=False,width=450)
a.xaxis.axis_label = ""
b.xaxis.axis_label = ""
show(row(a,b))
a = Bar(df, 'Name', values='Cholesterol (mg)', title="Excess Cholesterol",legend=False,width=450)
b = Bar(df, 'Name', values='Saturated Fat (g)', title="Excess Saturated Fats",legend=False,width=450)
a.xaxis.axis_label = ""
b.xaxis.axis_label = ""
show(row(a,b))
a = Bar(df, 'Name', values='Sodium (mg)', title="Excess Sodium",legend=False,width=450)
b = Bar(df, 'Name', values='Potassium (mg)', title="Excess Potassium",legend=False,width=450)
a.xaxis.axis_label = ""
b.xaxis.axis_label = ""
show(row(a,b))
a = Bar(df, 'Name', values='Carbohydrates (g)', title="Excess Carbohydrates",legend=False,width=450)
b = Bar(df, 'Name', values='Fiber (g)', title="Excess Fiber",legend=False,width=450)
a.xaxis.axis_label = ""
b.xaxis.axis_label = ""
show(row(a,b))
output_notebook()

Loading BokehJS ...

Like before, let's look at these as a percentage of the 2000 calorie diet.

In [13]:
def findPercentOveragesCal(food):
    for x in range(0,len(calnutrients)):
        nutrient = calnutrients[x]
        rec = recommended[x]
        if(rec == -1):
            continue
        food[nutrient] = ((food[nutrient]/rec)*100)-100
    return food   
caloverage_foods = calweighted_foods.apply(findPercentOveragesCal,axis=1)
caloverage_foods[display].sort_values(by='Calories').head(10).rename(index=str,columns={'Protein (g)':'Protein (%)','Total Fat(g)':'Total Fat(%)','Cholesterol (mg)':'Cholesterol (%)',
               'Saturated Fat (g)':'Saturated Fat (%)','Sodium (mg)':'Sodium (%)','Potassium (mg)':'Potassium (%)',
               'Carbohydrates (g)':'Carbohydrates (%)','Fiber (g)':'Fiber (g)'})

,Name,Calories,Protein (%),Total Fat(%),Cholesterol (%),Saturated Fat (%),Sodium (%),Potassium (%),Carbohydrates (%),Fiber (g)
NDB_No,,,,,,,,,,
3127,"BABYFOOD,VEG,SPINACH,CRMD,STR",2220.000000,200.000000,20.000000,0.000000,110.600000,22.500000,227.428571,14.000000,332.000000
6082,"CAMPBELL'S RED & WHITE,CHICK NOODLEO'S SOUP,COND",2330.808081,56.262626,0.000000,75.084175,30.328283,421.148990,302.380952,30.218855,5.050505
16059,"CHILI WITH BEANS,CANNED",2333.836858,177.341390,31.071346,28.398792,28.361027,299.358006,136.296936,0.000000,199.093656
6986,"CAMPBELL'S HEALTHY REQUEST,CHICK NOODLE SOUP,COND",2350.000000,140.000000,0.000000,66.666667,0.000000,595.833333,400.000000,8.500000,100.000000
22982,"KASHI,STEAM MEAL,CHICK FETTUCCINE,FRZ ENTREE",2383.333333,198.518519,0.000000,12.345679,8.333333,66.512346,15.555556,12.345679,92.592593
11372,"POTATOES,SCALLPD,HOME-PREPARED W/BUTTER",2448.979592,59.740260,57.556729,11.317254,213.775510,288.450835,200.556586,0.000000,111.502783
11387,"POTATOES,SCALLPD,DRY MIX,PREP W/H2O,WHL MILK&B...",2536.363636,15.636364,80.419580,0.000000,259.045455,287.500000,58.181818,16.090909,20.000000
3296,"BABYFOOD,TURKEY,RICE&VEG,TODD",2571.428571,225.714286,5.494505,0.000000,7.142857,226.785714,31.020408,7.142857,37.142857
11385,"POTATOES,AU GRATIN,DRY MIX,PREP W/H2O,WHL MILK...",2583.333333,27.777778,76.068376,38.888889,259.166667,408.101852,73.809524,18.888889,0.000000


Charting again with a uniform scale:

In [14]:
df = caloverage_foods.sort_values(by='Calories').head(10).rename(index=str,columns={'Protein (g)':'Protein (%)','Total Fat(g)':'Total Fat(%)','Cholesterol (mg)':'Cholesterol (%)',
               'Saturated Fat (g)':'Saturated Fat (%)','Sodium (mg)':'Sodium (%)','Potassium (mg)':'Potassium (%)',
               'Carbohydrates (g)':'Carbohydrates (%)','Fiber (g)':'Fiber (g)'})
a = Bar(df, label='vars',group='Name', 
        values=blend('Protein (%)', 'Total Fat(%)','Cholesterol (%)',
                     'Saturated Fat (%)','Sodium (%)','Potassium (%)',
                     'Carbohydrates (%)','Fiber (%)',name='values', labels_name='vars'),
        title="Excess Nutrients (% above recommended daily intake)",width=900)
a.xaxis.axis_label = ""
a.yaxis.axis_label = "% above reccomended daily intake"
show(a)
output_notebook()

Loading BokehJS ...

Again, our sodium and potassium are much higher than the other nutrients. Let's take them out and take another look.

In [15]:
df = caloverage_foods.sort_values(by='Calories').head(10).rename(index=str,columns={'Protein (g)':'Protein (%)','Total Fat(g)':'Total Fat(%)','Cholesterol (mg)':'Cholesterol (%)',
               'Saturated Fat (g)':'Saturated Fat (%)','Sodium (mg)':'Sodium (%)','Potassium (mg)':'Potassium (%)',
               'Carbohydrates (g)':'Carbohydrates (%)','Fiber (g)':'Fiber (g)'})
a = Bar(df, label='vars',group='Name', 
        values=blend('Protein (%)', 'Total Fat(%)','Cholesterol (%)',
                     'Saturated Fat (%)',
                     'Carbohydrates (%)','Fiber (%)',name='values', labels_name='vars'),
        title="Excess Nutrients (% above recommended daily intake) (Excluding Sodium & Potassium)",width=900,height=1000,palette=palettes.BrBG11)
a.xaxis.axis_label = ""
a.yaxis.axis_label = "% above reccomended daily intake"
show(a)
output_notebook()

Loading BokehJS ...

While these percentages aren't nearly as high as when looking at the overages for weight, we can still see a lot of overages for per calorie nutrients. However, it seems now that instead of sacrificing other nutrients for carbs, it is sacrificing them for cholesterol.

## What's in your cart?

Let's look at the average of food nutrients found in this dataset, and see if there are any weird things happening...
THINGS TO ADD TO THIS - Maybe show recommended nutrient intake stacked against this

In [16]:
print avg_nutrients
avg_nutrients["Sodium (mg)"] /= 1000
avg_nutrients["Cholesterol (mg)"] /= 1000
avg_nutrients["Potassium (mg)"] /= 1000
p = Bar(avg_nutrients)
show(p)

Protein (g)          0.113428
Total Fat(g)         0.105537
Cholesterol (mg)     0.387189
Saturated Fat (g)    0.034343
Sodium (mg)          3.095452
Potassium (mg)       2.659283
Carbohydrates (g)    0.221277
Fiber (g)            0.020396
dtype: float64
